In [ ]:
# Plotly and Dash library
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go
import plotly.express as px
import plotly

# Arrays and Dataframe
from collections import deque  # An array that can contain a certain amount of values
                               # it deletes automatically the oldest
import pandas as pd
import numpy as np

# Other
import random
import json
import ast

# Custom
from dashboard_utils import *

In [ ]:
# Stuff for the plot (UP LEFT)
index = [0]

totalcount_X = deque(maxlen=10)
totalcount_X.append(0)
totalcount = deque(maxlen=10)
totalcount.append(0)


# The following variables are needed for the 'cumulative feature'

# Counts for the cumulative histograms, we set a fixed number of bins (more than we need !memory wasted!)
# for a easier coding (I'd change it if I find a good solution)
NBINS = 200
tab5counts = [[0]*NBINS, [0]*NBINS, [0]*NBINS, [0]*NBINS]
tab6counts = [[0]*NBINS, [0]*NBINS, [0]*NBINS, [0]*NBINS]
tab7counts = [[0]*NBINS, [0]*NBINS, [0]*NBINS, [0]*NBINS]
tab8counts = [[0]*NBINS, [0]*NBINS, [0]*NBINS, [0]*NBINS]

tabscounts = [tab5counts,tab6counts,tab7counts,tab8counts] # Just the array of the array of counts

# We need to set the bins too, they will be defined when loading the message file,
# but we need a dummy array in case there's no file to load
tab5_bins = [0,1]
tab6_bins = [0,1]
tab7_bins = [0,1]
tab8_bins = [0,1]

tabsbins = [tab5_bins,tab6_bins ,tab7_bins ,tab8_bins] # Just the array of the array of bins

# For each figure we need to pay attention that the program does not update the counts on every callback, but
# only when the file is updated, using just a index messes with the callback and the counts are updated multiple times.
# This is the only solution I came with so far
indeces = [[[0],[0],[0],[0]],[[0],[0],[0],[0]],[[0],[0],[0],[0]],[[0],[0],[0],[0]]]

In [ ]:
'''Gets the data from file and creates the table (UP RIGHT)'''
def counttable_getdata():
    message = load_obj('message') # Load the dictionary
    # Update the table from a dataframes using dictionaries
    df = pd.DataFrame()
    data = {'ch': 'All', 'counts': message['Total Count']}
    df = df.append(data, ignore_index=True)
    for i in range(4):
        data = {'ch': 'Chamber '+str(i), 'counts':message['Chamber_'+str(i+1)]['Count']}
        df = df.append(data, ignore_index=True)
    
    return df.to_dict('records')

''' Load from the files the necessary data to build the live histogram'''
def hist_getdata(numhist,chamber):
    message = load_obj('message') # Load the dictionary
    
    # Load the data from the dictionary
    data = {'y': message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Counts'],
            'x': message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Bins'][1:],
            'type':'bar'}
    
    return {'data': [data], 'layout' : go.Layout(title='Chamber '+str(chamber - 1),
                                                 yaxis=dict(title=titleaxes[numhist-1][1],
                                                            range=[0,getmaxcount(message,numhist)]),
                                                 xaxis=dict(title=titleaxes[numhist-1][0],
                                                            range=[min(message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Bins'][1:]),
                                                                   max(message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Bins'][1:])]))}

'''Load from the file and update the arrays for the cumulative histograms'''
def cumulative_hist_getdata(numhist,chamber):
    # This variables will be updated, we need them to be globals
    global tabscounts
    global tabsbins
    
    message = load_obj('message') # Load the dictionary
    
    # Check if we already loaded the data for each figure
    if message['Index'] != indeces[numhist-1][chamber - 1][-1]:
        indeces[numhist-1][chamber - 1].append(message['Index']) # If not this grats us not to read it again
        
        # Updates the counts
        tabscounts[numhist-1][chamber - 1] = [sum(x) for x in zip(tabscounts[numhist-1][chamber - 1], message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Counts'])]
        tabsbins[numhist-1] = message['Chamber_'+str(chamber)]['Hist_'+str(numhist)]['Bins'][1:]
    
    data = {'y': tabscounts[numhist-1][chamber -1],
            'x': tabsbins[numhist-1],
            'type':'bar'}
    
    return {'data': [data], 'layout' : go.Layout(title='Chamber '+str(chamber - 1),
                                                 yaxis=dict(title=titleaxes[numhist-1][1],
                                                            range=[0,cumulative_getmaxcount(tabscounts[numhist-1])]),
                                                 xaxis=dict(title=titleaxes[numhist-1][0],
                                                            range=[min(tabsbins[numhist-1]),
                                                                   max(tabsbins[numhist-1])]))}

In [ ]:
'''TABS'''
# General
tabs_styles = {'height': '44px'}

# The tab
tab_style = {
                'borderBottom': '1px solid #d6d6d6',
                'padding': '6px',
                'fontWeight': 'bold'
            }

# Extra tab
extra_tab_style1 = {
                        'borderBottom': '1px solid #FFDC00 ',
                        'borderTop': '1px solid #FFDC00 ',
                        'borderLeft': '1px solid #FFDC00 ',
                        'padding': '6px',
                        'fontWeight': 'bold'
                   }

extra_tab_style2 = {
                        'borderBottom': '1px solid #FFDC00 ',
                        'borderTop': '1px solid #FFDC00 ',
                        'borderRight': '1px solid #FFDC00 ',
                        'padding': '6px',
                        'fontWeight': 'bold'
                   }

# Tab when active
tab_selected_style = {'padding': '6px','fontWeight': 'bold'}

extra_tab_selected_style = {'padding': '6px','fontWeight': 'bold','borderTop': '2px solid #FFDC00 '}

# Columns name (for creating the dataframe and stuff)
tblcols=[{'name': 'Chamber', 'id': 'ch'},
         {'name': 'Counts', 'id': 'counts'}]

'''HISTOGRAMS''' 
# Container
hist_div_style = {'display': 'inline-block', 'width':'50%'}

# The histogram
hist_style = {'height':'20vw'}

'''TABLE'''
# Header
table_header_style = {'fontWeight': 'default'}

# The table
table_style = {'width': '180%'}

# Cells
table_cell_style = {'textAlign': 'left', 'fontSize': '120%'}

# Conditional [if the column is the first, we bold it]
table_conditional_style = [{
                            'if': {'column_id': 'ch'},
                            'fontWeight': 'bold'
                          }]

titleaxes = [['TDC_CHANNEL','Counts'],
             ['Active TDC_CHANNEL','ORBIT_CNT'],
             ['Active TDC_CHANNEL','Scint. ORBIT_CNT'],
             ['DRIFTIME','Counts']]

In [ ]:
app = dash.Dash(__name__)

# LAYOUT OF THE WEB PAGE
app.layout = html.Div(
    [   # TITLE
        html.H2('Live Dashboard 🔴'),
        # to make it side by side you need both elements to be in children=[,] and have style={'display': 'inline-block'}
        html.Div(children=[
            # Container of Graph
            html.Div(dcc.Graph(id='live-graph', animate=True),
                     style={'width': '60%', 'display': 'inline-block',"margin-right": "5%"}),
            # Container of Table
            html.Div(dash_table.DataTable(id = 'count_table', data = counttable_getdata(), columns=tblcols,
                                          style_header=
                                            {
                                                'fontWeight': 'default'
                                            },
                                          style_table={'width': '180%'},
                                          style_cell={'textAlign': 'left', 'fontSize': '120%'},
                                          style_cell_conditional=[
                                            {
                                                'if': {'column_id': 'ch'},
                                                'fontWeight': 'bold'
                                            }]),style={'display': 'inline-block',"verticalAlign": "top",
                                                       "margin-top": "5%"})
            
        ]),
        html.Div([
            dcc.Tabs(id='cumulavornot', value='active', children =[
                dcc.Tab(label='Single Instance', value='active', style=tab_style, selected_style=tab_selected_style,
                        children=[
                            dcc.Tabs(id='tabs', value='active', children=[
                                    dcc.Tab(label='Histogram 1', value='active', style=tab_style, selected_style=tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist1-1', animate=True,
                                                                figure = hist_getdata(1,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist1-2', animate=True,
                                                                figure = hist_getdata(1,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist1-3', animate=True,
                                                                figure = hist_getdata(1,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist1-4', animate=True,
                                                                figure = hist_getdata(1,4), 
                                                                style=hist_style),style=hist_div_style)
                                                     ]),
                                    dcc.Tab(label='Histogram 2', value='tab-2', style=tab_style, selected_style=tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist2-1', animate=True,
                                                                figure = hist_getdata(2,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist2-2', animate=True,
                                                                figure = hist_getdata(2,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist2-3', animate=True,
                                                                figure = hist_getdata(2,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist2-4', animate=True,
                                                                figure = hist_getdata(2,4), 
                                                                style=hist_style),style=hist_div_style)
                                                    ]),
                                    dcc.Tab(label='EXTRA 1', value='tab-3', style=extra_tab_style1, selected_style=extra_tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist3-1', animate=True,
                                                                figure = hist_getdata(3,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist3-2', animate=True,
                                                                figure = hist_getdata(3,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist3-3', animate=True,
                                                                figure = hist_getdata(3,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist3-4', animate=True,
                                                                figure = hist_getdata(3,4), 
                                                                style=hist_style),style=hist_div_style)
                                                    ]),

                                    dcc.Tab(label='EXTRA 2', value='tab-4', style=extra_tab_style2, selected_style=extra_tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist4-1', animate=True,
                                                                figure = hist_getdata(4,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist4-2', animate=True,
                                                                figure = hist_getdata(4,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist4-3', animate=True,
                                                                figure = hist_getdata(4,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist4-4', animate=True,
                                                                figure = hist_getdata(4,4), 
                                                                style=hist_style),style=hist_div_style)
                                                    ])

                                ]),]),
                        ### CUMULATIVE TAB ###
                        dcc.Tab(label='Cumulative', value='cumulative', style=tab_style, selected_style=tab_selected_style,
                                children = [
                                    dcc.Tabs(id='cumulativetabs', value='tab-5', children=[
                                    dcc.Tab(label='Histogram 1', value='tab-5', style=tab_style, selected_style=tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist5-1', animate=True,
                                                                figure = cumulative_hist_getdata(1,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist5-2', animate=True,
                                                                figure = cumulative_hist_getdata(1,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist5-3', animate=True,
                                                                figure = cumulative_hist_getdata(1,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist5-4', animate=True,
                                                                figure = cumulative_hist_getdata(1,4), 
                                                                style=hist_style),style=hist_div_style)
                                                     ]),
                                    dcc.Tab(label='Histogram 2', value='tab-2', style=tab_style, selected_style=tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist6-1', animate=True,
                                                                figure = cumulative_hist_getdata(2,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist6-2', animate=True,
                                                                figure = cumulative_hist_getdata(2,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist6-3', animate=True,
                                                                figure = cumulative_hist_getdata(2,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist6-4', animate=True,
                                                                figure = cumulative_hist_getdata(2,4), 
                                                                style=hist_style),style=hist_div_style)
                                                    ]),
                                    dcc.Tab(label='EXTRA 1', value='tab-3', style=extra_tab_style1, selected_style=extra_tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist7-1', animate=True,
                                                                figure = cumulative_hist_getdata(3,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist7-2', animate=True,
                                                                figure = cumulative_hist_getdata(3,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist7-3', animate=True,
                                                                figure = cumulative_hist_getdata(3,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist7-4', animate=True,
                                                                figure = cumulative_hist_getdata(3,4), 
                                                                style=hist_style),style=hist_div_style)
                                                    ]),

                                    dcc.Tab(label='EXTRA 2', value='tab-4', style=extra_tab_style2, selected_style=extra_tab_selected_style,
                                            children=[html.Div(dcc.Graph(id = 'hist8-1', animate=True,
                                                                figure = cumulative_hist_getdata(4,1), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist8-2', animate=True,
                                                                figure = cumulative_hist_getdata(4,2), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist8-3', animate=True,
                                                                figure = cumulative_hist_getdata(4,3), 
                                                                style=hist_style),style=hist_div_style),
                                                      html.Div(dcc.Graph(id = 'hist8-4', animate=True,
                                                                figure = cumulative_hist_getdata(4,4), 
                                                                style=hist_style),style=hist_div_style)

])])])],)]),

        dcc.Interval(id='graph-update')]
)

# CALLBACK FOR COUNT GRAPH
@app.callback(Output('live-graph', 'figure'),
        [Input('graph-update', 'n_intervals')])
def update_graph_scatter(n):    
    message = load_obj('message')
    if message['Index'] != index[-1]:
        if len(totalcount_X) != 0:
            totalcount_X.append(totalcount_X[-1] + 1)
        else:
            totalcount_X.append(0)
            
        totalcount.append(message['Total Count'])
        
        index.append(message['Index'])

    data = plotly.graph_objs.Scatter(
            x=list(totalcount_X),
            y=list(totalcount),
            name='Scatter',
            mode= 'lines+markers'
            )
    if len(totalcount_X) == 1:
        return {'data': [data],'layout' : go.Layout(height=300,
                                                    title='Total Count of Events')}
    else:
        return {'data': [data],'layout' : go.Layout(height=300,
                                                    title='Total Count of Events',
                                                    xaxis=dict(title='Time step',
                                                               range=[min(totalcount_X),max(totalcount_X)]),
                                                    yaxis=dict(title='Counts',
                                                               range=[min(totalcount),max(totalcount)]),)}

# CALLBACK FOR TABLE
@app.callback(Output('count_table','data'),
             [Input('graph-update', 'n_intervals')])
def updateTable(n):
     return counttable_getdata()
    
# CALLBACK FOR HIST

### HIST 1 ###
@app.callback(Output('hist1-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist1(n):
     return hist_getdata(1,1)
    
@app.callback(Output('hist1-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist2(n):
     return hist_getdata(1,2)
    
@app.callback(Output('hist1-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist3(n):
     return hist_getdata(1,3)
    
@app.callback(Output('hist1-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist4(n):
     return hist_getdata(1,4)
@app.callback(Output('hist2-1','figure'),
             [Input('graph-update', 'n_intervals')])

### HIST 2 ###
def updateHist5(n):
     return hist_getdata(2,1)
    
@app.callback(Output('hist2-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist6(n):
     return hist_getdata(2,2)
    
@app.callback(Output('hist2-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist7(n):
     return hist_getdata(2,3)
    
@app.callback(Output('hist2-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist8(n):
     return hist_getdata(2,4)

### EXTRA 1 ###
@app.callback(Output('hist3-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist9(n):
    return hist_getdata(3,1)
    
@app.callback(Output('hist3-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist10(n):
     return hist_getdata(3,2)
    
@app.callback(Output('hist3-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist11(n):
     return hist_getdata(3,3)
    
@app.callback(Output('hist3-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist12(n):
     return hist_getdata(3,4)


### EXTRA 2 ###
@app.callback(Output('hist4-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist13(n):
     return hist_getdata(4,1)
    
@app.callback(Output('hist4-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist14(n):
     return hist_getdata(4,2)
    
@app.callback(Output('hist4-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist15(n):
     return hist_getdata(4,3)
    
@app.callback(Output('hist4-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist16(n):
     return hist_getdata(4,4)
    
## CUMULATIVE ##
### HIST 1 ###
@app.callback(Output('hist5-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist17(n):
     return cumulative_hist_getdata(1,1)
    
@app.callback(Output('hist5-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist18(n):
     return cumulative_hist_getdata(1,2)
    
@app.callback(Output('hist5-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist19(n):
     return cumulative_hist_getdata(1,3)
    
@app.callback(Output('hist5-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist20(n):
     return cumulative_hist_getdata(1,4)

### HIST 2 ###    
@app.callback(Output('hist6-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist21(n):
     return cumulative_hist_getdata(2,1)
    
@app.callback(Output('hist6-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist22(n):
     return cumulative_hist_getdata(2,2)
    
@app.callback(Output('hist6-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist23(n):
     return cumulative_hist_getdata(2,3)
    
@app.callback(Output('hist6-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist24(n):
     return cumulative_hist_getdata(2,4)

### EXTRA 1 ###
@app.callback(Output('hist7-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist25(n):
     return cumulative_hist_getdata(3,1)
    
@app.callback(Output('hist7-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist26(n):
     return cumulative_hist_getdata(3,2)
    
@app.callback(Output('hist7-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist27(n):
     return cumulative_hist_getdata(3,3)
    
@app.callback(Output('hist7-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist28(n):
     return cumulative_hist_getdata(3,4)

### EXTRA 2 ###
@app.callback(Output('hist8-1','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist29(n):
     return cumulative_hist_getdata(4,1)
    
@app.callback(Output('hist8-2','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist30(n):
     return cumulative_hist_getdata(4,2)
    
@app.callback(Output('hist8-3','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist31(n):
     return cumulative_hist_getdata(4,3)
    
@app.callback(Output('hist8-4','figure'),
             [Input('graph-update', 'n_intervals')])
def updateHist32(n):
     return cumulative_hist_getdata(4,4)
    
    
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)
